<a href="https://colab.research.google.com/github/codershiva91/Email-Marketing-ML-model/blob/main/Email_Marketing_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets Import all t he necessary Libaraies

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
